## tfrs based explicit
_tfrs based explicit recommendation model_ \
[Source](https://www.tensorflow.org/recommenders/examples/basic_ranking) | [Dataset](https://files.grouplens.org/datasets/movielens/ml-1m.zip) \
Model trained under the RMSE loss to regress the rating given bu the user for the certain items. The model could used as both recommendation model and ranking model.\
_*Explicit rating dataset had to be used_

In [4]:
import os
import pprint
import tempfile

from typing import Dict, Text

import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

#### Data preparations

In [5]:
rating_dat = pd.read_csv('../data/datasets/ml-1m/ratings.dat', header=None, sep='::', engine='python')
rating_dat.columns = ['UserId', 'MovieId', 'Rating', 'Timestamp']
rating_dat.head()

,UserId,MovieId,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [7]:
item_dat = pd.read_csv('../data/datasets/ml-1m/movies.dat', header=None, sep='::', engine='python', encoding = "ISO-8859-1")
item_dat.columns = ['MovieId', 'Title', 'Genres']
item_dat.head()

,MovieId,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [9]:
user_tensor = tf.strings.as_string(rating_dat.UserId.values)
rating_tensor = tf.convert_to_tensor(rating_dat.Rating.values)
movie_title_tensor = tf.strings.as_string(rating_dat.MovieId.values)

rating_dt = tf.data.Dataset.from_tensor_slices({'movie_title': movie_title_tensor, 'user_id': user_tensor, 'user_rating': rating_tensor})

In [10]:
tf.random.set_seed(42)
shuffled = rating_dt.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [11]:
movie_titles = rating_dt.batch(10_000).map(lambda x: x["movie_title"])
user_ids = rating_dt.batch(10_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

#### Model building

In [12]:
class RankingModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    embedding_dimension = 32

    # Compute embeddings for users.
    self.user_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # Compute embeddings for movies.
    self.movie_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
    ])

    # Compute predictions.
    self.ratings = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
  ])

  def call(self, inputs):

    user_id, movie_title = inputs

    user_embedding = self.user_embeddings(user_id)
    movie_embedding = self.movie_embeddings(movie_title)

    return self.ratings(tf.concat([user_embedding, movie_embedding], axis=1))

In [15]:
class MovielensModel(tfrs.models.Model):

  def __init__(self):
    super().__init__()
    self.ranking_model = RankingModel()
    self.task = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def call(self, features):
    return self.ranking_model(
        (features["user_id"], features["movie_title"]))

  def compute_loss(self, features, training=False):
    labels = features.pop("user_rating")
    rating_predictions = self(features)

    return self.task(labels=labels, predictions=rating_predictions)

In [16]:
model = MovielensModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [17]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [18]:
model.fit(cached_train, epochs=3)

Epoch 1/3
10/10 [==============================] - 4s 33ms/step - root_mean_squared_error: 2.2389 - loss: 4.5882 - regularization_loss: 0.0000e+00 - total_loss: 4.5882
Epoch 2/3
10/10 [==============================] - 0s 14ms/step - root_mean_squared_error: 1.1105 - loss: 1.2316 - regularization_loss: 0.0000e+00 - total_loss: 1.2316
Epoch 3/3
10/10 [==============================] - 0s 14ms/step - root_mean_squared_error: 1.1061 - loss: 1.2217 - regularization_loss: 0.0000e+00 - total_loss: 1.2217


#### Model evaluation

In [19]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 1s 40ms/step - root_mean_squared_error: 1.1103 - loss: 1.2329 - regularization_loss: 0.0000e+00 - total_loss: 1.2329


{'root_mean_squared_error': 1.110325813293457,
 'loss': 1.2332055568695068,
 'regularization_loss': 0,
 'total_loss': 1.2332055568695068}

#### Model prediction

In [20]:
test_ratings = {}
test_movie_titles = ["M*A*S*H (1970)", "Dances with Wolves (1990)", "Speed (1994)"]
for movie_title in test_movie_titles:
  test_ratings[movie_title] = model({
      "user_id": np.array(["42"]),
      "movie_title": np.array([movie_title])
  })

print("Ratings:")
for title, score in sorted(test_ratings.items(), key=lambda x: x[1], reverse=True):
  print(f"{title}: {score}")

Ratings:
M*A*S*H (1970): [[3.5119967]]
Dances with Wolves (1990): [[3.5119967]]
Speed (1994): [[3.5119967]]


#### Model saving & loading

In [21]:
tf.saved_model.save(model, '../tmp/models/')

2022-06-04 14:57:22.604016: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ../tmp/models/assets


INFO:tensorflow:Assets written to: ../tmp/models/assets


In [22]:
loaded = tf.saved_model.load('../tmp/models/')

loaded({"user_id": np.array(["42"]), "movie_title": ["Speed (1994)"]}).numpy()

array([[3.5119967]], dtype=float32)